In [1]:
from read import read_voter_file, read_score_file, read_order_file
from tools import all_mass_init, flatten_pref_mass_mat
from beliefKMeans import k_means, _k_means_single, silhouette_score

import numpy as np

[1546, 1535, 1406, 1457, 1380, 1447, 1412, 1353, 1321, 1244, 1297, 1213, 1194, 1118, 1112, 1077, 1012, 1016, 968, 957, 939, 945, 868, 857, 841, 847, 785, 775, 699, 694, 696, 619, 512, 567, 560, 545, 500, 532, 509, 453, 433, 468, 413, 334, 336, 327, 358, 366, 347, 348, 355, 385, 287, 282, 231, 199, 214, 218, 219, 236, 219, 234, 213, 215, 234, 155, 142, 138, 135, 155, 141, 164, 169, 133, 165, 148, 144, 131, 150, 153, 78, 67, 88, 50, 66, 80, 52, 77, 85, 84, 74, 80, 76, 75, 72, 95, 68, 79, 75, 77]
[0, 1, 3, 5, 6, 2, 4, 7, 8, 10, 9, 11, 12, 13, 14, 15, 17, 16]
1
[3182]
[[ 1.   0.   0.   0. ]
 [ 0.   1.   0.   0.5]
 [ 0.   0.   1.   0.5]
 [ 0.   0.5  0.5  1. ]]
center:[  1.85037171e-17   4.66666667e-01   2.33333333e-01   3.00000000e-01]


Open user information data, set score and order data into it

In [17]:
vMap = read_voter_file("../data/sushi3-2016/sushi3.com30.udata")
read_score_file("../data/sushi3-2016/sushi3b.com30.10.score", vMap)
read_order_file("../data/sushi3-2016/sushi3b.com30.10.order", vMap)
# candidate list for 40 cand
"""
cand_list=[0, 1, 3, 5, 6, 2, 4, 7, 8, 10, 
           9, 11, 12, 13, 14, 15, 17, 16, 
           18, 19, 21, 20, 22, 23, 25, 24, 
           26, 27, 28, 30, 29, 31, 33, 34, 
           35, 37, 32, 38, 36, 41]
#candidate list for 35 cand

"""
"""
cand_list=[0, 1, 3, 5, 6, 2, 4, 7, 8, 10, 
           9, 11, 12, 13, 14, 15, 17, 16, 
           18, 19, 21, 20, 22, 23, 25, 24, 
           26, 27, 28, 30, 29, 31, 33, 34, 35]
"""
#candidate list for 30 cands

cand_list = [0, 1, 3, 5, 6, 2, 4, 7, 8, 10,
             9, 11, 12, 13, 14, 15, 17, 16, 
             18, 19, 21, 20, 22, 23, 25,24, 
             26, 27, 28, 30,]

#cand_list = list(range(100))
#n_item = 40
n_item = 30
#n_item = 100

In [2]:
vMap = read_voter_file("../data/sushi3-2016/artif_inverse_2.udata")
read_score_file("../data/sushi3-2016/artif_inverse_2.score", vMap)
read_order_file("../data/sushi3-2016/artif_inverse_2.order", vMap)
cand_list=[8, 0, 13, 1, 9, 17, 11, 10, 7, 5]
cand_list.sort()
n_item = 10

In [9]:
cand_list.sort()
print(cand_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 41]


Initialize the mass function value

In [18]:
n_sample = len(vMap)
vector_size = 16
n_pref_pair = int(n_item*(n_item-1)/2)
X = np.empty((n_sample, n_pref_pair, vector_size),dtype = object)


In [4]:
import sys
sys.getsizeof(X)/(1024**3)

0.0028003454208374023

In [19]:
for k,v in vMap.items():
    X[k] = flatten_pref_mass_mat(all_mass_init(v.get_scores(), v.get_order_b(), cand_list, nbItem = n_item))

In [5]:
X.shape

(147, 595, 16)

In [6]:
from beliefDBSCAN import BeliefDBSCAN
cluster_model = BeliefDBSCAN(min_samples=5, eps=4)
cluster_model.fit(X)

In [6]:
cluster_model.distances

array([[ 0.        ,  0.        ,  0.69354328, ...,  0.15257284,
         2.58030666,  1.95358925],
       [ 0.        ,  0.        ,  0.82294885, ...,  1.87626399,
         2.64189203,  1.07279211],
       [ 0.69354328,  0.82294885,  0.        , ...,  1.42731784,
         1.50998855,  1.76199057],
       ..., 
       [ 0.15257284,  1.87626399,  1.42731784, ...,  0.        ,
         0.57376535,  0.46875333],
       [ 2.58030666,  2.64189203,  1.50998855, ...,  0.57376535,
         0.        ,  1.25333333],
       [ 1.95358925,  1.07279211,  1.76199057, ...,  0.46875333,
         1.25333333,  0.        ]])

In [7]:
hasattr(cluster_model, "distances")

True

In [20]:
for eps in np.arange(0.1,2,0.1):
    for minPts in range(1, 6):
        cluster_model.eps = eps
    #cluster_model.min_samples = minPts
        labels=cluster_model.predict(X)
        print(eps,minPts, silhouette_score(X,labels, metric = "conflict", distances=cluster_model.distances))

/home/yzhang/ThesisWork/Program/sushiPref/sushiCode/sushiCode/beliefKMeans.py:560: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


0.1 1 nan
0.1 2 nan
0.1 3 nan
0.1 4 nan
0.1 5 nan
0.2 1 -0.405237048964
0.2 2 -0.405237048964
0.2 3 -0.405237048964
0.2 4 -0.405237048964
0.2 5 -0.405237048964
0.3 1 nan
0.3 2 nan
0.3 3 nan
0.3 4 nan
0.3 5 nan
0.4 1 -0.45466348176
0.4 2 -0.45466348176
0.4 3 -0.45466348176
0.4 4 -0.45466348176
0.4 5 -0.45466348176
0.5 1 -0.374383943358
0.5 2 -0.374383943358
0.5 3 -0.374383943358
0.5 4 -0.374383943358
0.5 5 -0.374383943358
0.6 1 -0.467962256365
0.6 2 -0.467962256365
0.6 3 -0.467962256365
0.6 4 -0.467962256365
0.6 5 -0.467962256365
0.7 1 -0.524465162796
0.7 2 -0.524465162796
0.7 3 -0.524465162796
0.7 4 -0.524465162796
0.7 5 -0.524465162796
0.8 1 -0.265294573137
0.8 2 -0.265294573137
0.8 3 -0.265294573137
0.8 4 -0.265294573137
0.8 5 -0.265294573137
0.9 1 -0.280579547388
0.9 2 -0.280579547388
0.9 3 -0.280579547388
0.9 4 -0.280579547388
0.9 5 -0.280579547388
1.0 1 -0.364537864538
1.0 2 -0.364537864538
1.0 3 -0.364537864538
1.0 4 -0.364537864538
1.0 5 -0.364537864538
1.1 1 -0.180665076358
1.1

In [21]:
cluster_model.eps = 1.6
cluster_model.min_samples = 3
labels = cluster_model.predict(X)

In [32]:
for k in np.where(labels==3)[0]:
    print(k)
    print(vMap[k].get_scores())
    print(vMap[k].get_order_b())

20
{0: 3, 1: 3, 2: 3, 5: 4, 6: 4, 17: 2, 8: 1, 12: 4, 24: 4, 21: 4}
[12, 24, 6, 21, 0, 1, 5, 2, 8, 17]
38
{0: 2, 1: 2, 26: 4, 4: 0, 16: 4, 7: 4, 8: 4, 10: 4, 23: 2, 13: 4}
[10, 8, 13, 16, 0, 26, 7, 1, 23, 4]
43
{0: 4, 1: 4, 3: 3, 22: 2, 6: 4, 25: 2, 11: 4, 27: 4, 13: 4, 15: 3}
[11, 13, 1, 0, 6, 15, 27, 3, 22, 25]
49
{18: 4, 3: 3, 22: 3, 6: 4, 23: 2, 8: 3, 9: 1, 10: 4, 11: 1, 12: 3}
[18, 6, 10, 8, 22, 3, 12, 23, 11, 9]


In [12]:
# dataset: com30
for eps in np.arange(1,2,0.1):
    for minPts in range(1, 6):
        cluster_model.eps = eps
        cluster_model.min_samples = minPts
        labels=cluster_model.predict(X)
        print(eps,minPts, silhouette_score(X,labels, metric = "conflict"))

IndexError: index 54 is out of bounds for axis 0 with size 54

In [9]:
#dataset com40
vMap = read_voter_file("../data/sushi3-2016/sushi3.com40.udata")
read_score_file("../data/sushi3-2016/sushi3b.com40.10.score", vMap)
read_order_file("../data/sushi3-2016/sushi3b.com40.10.order", vMap)
cand_list=[0, 1, 3, 5, 6, 2, 4, 7, 8, 10, 
           9, 11, 12, 13, 14, 15, 17, 16, 
           18, 19, 21, 20, 22, 23, 25, 24, 
           26, 27, 28, 30, 29, 31, 33, 34, 
           35, 37, 32, 38, 36, 41]
n_item = 40
n_sample = len(vMap)
vector_size = 16
n_pref_pair = int(n_item*(n_item-1)/2)
X = np.empty((n_sample, n_pref_pair, vector_size),dtype = object)
for k,v in vMap.items():
    X[k] = flatten_pref_mass_mat(all_mass_init(v.get_scores(), v.get_order_b(), cand_list, nbItem = n_item))
    


In [10]:
from beliefDBSCAN import BeliefDBSCAN
cluster_model40 = BeliefDBSCAN(min_samples=5, eps=4)
cluster_model40.fit(X)
print("model fitted")
for eps in np.arange(1,2,0.1):
    for minPts in range(1, 6):
        cluster_model40.eps = eps
        cluster_model40.min_samples = minPts
        labels=cluster_model40.predict(X)
        print(eps,minPts, silhouette_score(X,labels, metric = "conflict", distances = cluster_model40.distances))

model fitted
1.0 1 -0.549447730426
1.0 2 -0.549447730426
1.0 3 -0.549447730426
1.0 4 -0.549447730426
1.0 5 -0.549447730426
1.1 1 -0.405644296867
1.1 2 -0.405644296867
1.1 3 -0.405644296867
1.1 4 -0.405644296867
1.1 5 -0.405644296867
1.2 1 -0.616605366205
1.2 2 -0.616605366205
1.2 3 -0.616605366205
1.2 4 -0.616605366205
1.2 5 -0.616605366205
1.3 1 -0.366799889743
1.3 2 -0.366799889743
1.3 3 -0.366799889743
1.3 4 -0.366799889743
1.3 5 -0.366799889743
1.4 1 -0.377462900955
1.4 2 -0.377462900955
1.4 3 -0.377462900955
1.4 4 -0.377462900955
1.4 5 -0.377462900955
1.5 1 -0.372704565333
1.5 2 -0.372704565333
1.5 3 -0.372704565333
1.5 4 -0.372704565333
1.5 5 -0.372704565333
1.6 1 -0.369339135612
1.6 2 -0.369339135612
1.6 3 -0.369339135612
1.6 4 -0.369339135612
1.6 5 -0.369339135612
1.7 1 -0.0933471095854
1.7 2 -0.0933471095854
1.7 3 -0.0933471095854
1.7 4 -0.0933471095854
1.7 5 -0.0933471095854
1.8 1 -0.0896734399037
1.8 2 -0.0896734399037
1.8 3 -0.0896734399037
1.8 4 -0.0896734399037
1.8 5 -0.0

In [16]:
for eps in np.arange(2,3,0.1):
    for minPts in range(1, 6):
        cluster_model40.eps = eps
        cluster_model40.min_samples = minPts
        labels=cluster_model40.predict(X)
        print(eps,minPts, silhouette_score(X,labels, metric = "conflict", distances = cluster_model40.distances))

2.0 1 -0.0819098637693
2.0 2 -0.0819098637693
2.0 3 -0.0819098637693
2.0 4 -0.0819098637693
2.0 5 -0.0819098637693
2.1 1 -0.0838318518505
2.1 2 -0.0838318518505
2.1 3 -0.0838318518505
2.1 4 -0.0838318518505
2.1 5 -0.0838318518505
2.2 1 -0.405715601744
2.2 2 -0.405715601744
2.2 3 -0.405715601744
2.2 4 -0.405715601744
2.2 5 -0.405715601744
2.3 1 -0.104471857209
2.3 2 -0.104471857209
2.3 3 -0.104471857209
2.3 4 -0.104471857209
2.3 5 -0.104471857209
2.4 1 -0.0959980399848
2.4 2 -0.0959980399848
2.4 3 -0.0959980399848
2.4 4 -0.0959980399848
2.4 5 -0.0959980399848
2.5 1 -0.110512093741
2.5 2 -0.110512093741
2.5 3 -0.110512093741
2.5 4 -0.110512093741
2.5 5 -0.110512093741
2.6 1 -0.110512093741
2.6 2 -0.110512093741
2.6 3 -0.110512093741
2.6 4 -0.110512093741
2.6 5 -0.110512093741
2.7 1 -0.110512093741
2.7 2 -0.110512093741
2.7 3 -0.110512093741
2.7 4 -0.110512093741
2.7 5 -0.110512093741
2.8 1 -0.251315100153
2.8 2 -0.251315100153
2.8 3 -0.251315100153
2.8 4 -0.251315100153
2.8 5 -0.25131510

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [11]:
silhouette_score(X,labels)

-0.051084027597869723

In [7]:
labels, inertia, centers, distances, iterNb = _k_means_single(X, 2, init='k_means',max_iter = 20, verbose = True)
#centers, labels, inertia, distances, iterNb = k_means(X, 2,n_init=5, verbose=True, return_n_iter=True)

centers initialized, shape:(2, 595, 16)
Initialization complete
E step:
labels after step E: [1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1
 0 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0
 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1 0 1
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1]
M step:
[73 74]
new center for 0 calculated
new center for 1 calculated
Iteration  0, inertia 65.142 
Best inertia: None
best result updated
E step:
labels after step E: [1 0 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1
 0 0 0 1 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0
 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 1
 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1]
M step:
[77 70]
new center for 0 calculated
new center for 1 calculated
Iteration  1, inertia 66.097 
Best inertia: 65.1416665804
E step:
l

In [27]:
labels

array([1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0], dtype=int32)

In [15]:
np.bincount(labels, minlength=2)

array([2, 2])

In [16]:
inertia

0.0

In [10]:
centers

array([[[4.08903414923675e-18, 0.03559006211180057, 0.03613354037267074,
         ..., -1.9345636204093318e-15, 8.881784197001239e-17,
         0.912204968944101],
        [-1.9694391045528367e-18, 0.0456366459627337, 0.04472049689441075,
         ..., 3.423650252187823e-15, 4.08562073062057e-15,
         0.8817391304347761],
        [-2.1380412968018938e-18, 0.09032608695652238,
         0.027468944099379836, ..., 3.6706748751669214e-15,
         6.30606677987088e-15, 0.8716149068322897],
        ..., 
        [1.0915526591644531e-17, 0.005869565217391327,
         0.005341614906832341, ..., 2.386979502944088e-16,
         8.8817841970012395e-16, 0.9887888198757764],
        [2.2025514607477978e-17, 1.5462492682164045e-16,
         0.01074534161490708, ..., 1.4488410471358282e-15,
         2.3980817331903347e-15, 0.9892546583850903],
        [5.916456789157588e-32, -2.442689856871144e-16,
         -3.097886495062221e-17, ..., -8.743006318923093e-16, 0.0,
         1.000000000000001]],


In [8]:
from beliefKMeans import _inter_intra_inertia, _ratio_inter_intra_inertia
_inter_intra_inertia(centers, labels, distances)
_ratio_inter_intra_inertia(centers, labels, distances)

0.0092941482192552029

In [25]:
silhouette_score(X,labels)

/home/yzhang/ThesisWork/Program/sushiPref/sushiCode/sushiCode/iBelief/distance.py:18: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if D== None:


0.021218149852089679